# FFNN
This trains a classifier for the iris dataset.
Initially keras, matplotlib and numpy are imported

In [1]:
import keras
import matplotlib.pyplot as plt
import numpy as mp
import pandas as pd

Using TensorFlow backend.


## Opening the csv and splitting into inputs and outputs
Using pandas for the initial list import simple to see that just column 4 need be selected for the outputs

In [2]:
df = pd.read_csv('data.csv', header=None)
df


,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


## Splitting into inputs and ouptuts
Using a generator we canturn the output into a list of input and output tuples by using the `to_records` function inside the generators. Finally the list is sliced to provide the information requried. The data is initially shuffled so the validation split does not onl;y contain the end data.

In [3]:
df = df.sample(frac=1).reset_index(drop=True)
inputs = [tuple(x)[0:4] for x in df.to_records(index=False)]
outputs_raw = [tuple(x)[-1] for x in df.to_records(index=False)]

## Converting raw output into numeric data
Using another comprehension it is possible to map the strings into data values. A dictionary will be used for this purpose.

In [4]:
keys = {'Iris-setosa': (1,0,0), 'Iris-versicolor' : (0,1,0), 'Iris-virginica': (0,0,1)}
outputs = [keys[x] for x in outputs_raw]

# Starting model
Using the sequential model, create a fully connected 4 layer deep neural network. The `input_dim` needs to be set to the size of the inputs. In this case it is a linear size 4 input dimension and linear 3 size output dimension.

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential([(
    Dense(output_dim=7, input_dim=4)),
    Activation('relu'),
    Dense(output_dim=3)
                   ])

Using the stochastic gradient descent optimiser the MSE is used as the training value. It is then run for 1250 epochs using 20% of the data for validation.

In [6]:
from keras.optimizers import SGD
model.compile(loss='mean_squared_error', optimizer=SGD(lr=0.005, momentum=0.0, nesterov=False))
model.fit(inputs, outputs, nb_epoch=1250, batch_size=10, validation_split=0.2)

Train on 120 samples, validate on 30 samples
Epoch 1/1250
120/120 [==============================] - 0s - loss: 0.9815 - val_loss: 0.3724
Epoch 2/1250
120/120 [==============================] - 0s - loss: 0.3257 - val_loss: 0.2639
Epoch 3/1250
120/120 [==============================] - 0s - loss: 0.2665 - val_loss: 0.2469
Epoch 4/1250
120/120 [==============================] - 0s - loss: 0.2502 - val_loss: 0.2384
Epoch 5/1250
120/120 [==============================] - 0s - loss: 0.2410 - val_loss: 0.2307
Epoch 6/1250
120/120 [==============================] - 0s - loss: 0.2313 - val_loss: 0.2227
Epoch 7/1250
120/120 [==============================] - 0s - loss: 0.2221 - val_loss: 0.215

# Outputs
Using the `model.predict` function and comparing that to the initial data it is easy to see that the algorithm has either successfully or unsuccessfully trained the NN.

In [7]:
predict = [(int(x[0]), int(x[1]), int(x[2])) for x in model.predict(inputs)]
predict

[(0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (1, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (1, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (1, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 1),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (1, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (1, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (1, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (1, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0, 0, 0),
 (0,

In [8]:
[predict[i] == outputs[i] for i in range(len(predict))]

[False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False